Convert the CSV file containg the data for the participations to a netCDF file to be later used in DINEOF.

In [1]:
import netCDF4
import os
import numpy as np
import liegecolloquium

In [24]:
datadir = "../data/"
outputdir = "../data/dineof/"
datafile = os.path.join(datadir, "numParticipant.csv")
outputfile = os.path.join(outputdir, "ColloquiumXYTnoise.nc")
outputfile2 = os.path.join(outputdir, "ColloquiumTYXnoise.nc")

Read values from the CSV file

In [25]:
Year, Participants, Papers, Abstracts, Countries = liegecolloquium.get_clq_list(datafile)

Create 3D arrays for the different time series to simulate spacial coverage

In [26]:
Nx, Ny = 20, 10
papers_array = np.tile(np.array(Papers), (Nx, Ny, 1)) + np.random.rand(Nx, Ny, len(Year)) - 0.5
country_array = np.tile(np.array(Countries), (Nx, Ny, 1)) + np.random.rand(Nx, Ny, len(Year)) - 0.5
abstract_array = np.tile(np.array(Abstracts), (Nx, Ny, 1)) + np.random.rand(Nx, Ny, len(Year)) - 0.5
participant_array = np.tile(np.array(Participants), (Nx, Ny, 1)) + np.random.rand(Nx, Ny, len(Year)) - 0.5

In [29]:
papers_array[papers_array<-900] = valex
country_array[country_array<-900] = valex
abstract_array[abstract_array<-900] = valex
participant_array[participant_array<-900] = valex

Write into netCDF

In [30]:
valex = -999
with netCDF4.Dataset(outputfile, "w") as nc:
    time = nc.createDimension("time", None)
    x = nc.createDimension("x", Nx)
    y = nc.createDimension("y", Ny)
    
    lon = nc.createVariable("lon","f4",("x",))
    lat = nc.createVariable("lat","f4",("y",))
    
    year = nc.createVariable("year","f8",("time"))
    mask = nc.createVariable("mask","f8",("x", "y"))
    paper = nc.createVariable("paper","f8",("x", "y", "time"))
    country = nc.createVariable("country","f8",("x", "y", "time"))
    abstract = nc.createVariable("abstract","f8",("x", "y", "time"))
    participant = nc.createVariable("participant","f8",("x", "y", "time"))
    
    country.missing_value = valex
    paper.missing_value = valex
    abstract.missing_value = valex
    participant.missing_value = valex
    
    lon = np.linspace(0, 10, Nx)
    lat = np.linspace(0, 10, Ny)
    year[:] = np.array(Year)
    paper[:] = papers_array
    country[:] = country_array
    abstract[:] = abstract_array
    participant[:] = participant_array
    mask[:] = np.ones((Nx, Ny))
    #mask[:] = np.ones(len(Year))                                 

In [31]:
with netCDF4.Dataset(outputfile2, "w") as nc:
    time = nc.createDimension("time", None)
    x = nc.createDimension("x", Nx)
    y = nc.createDimension("y", Ny)
    
    lon = nc.createVariable("lon","f4",("x",))
    lat = nc.createVariable("lat","f4",("y",))
    
    year = nc.createVariable("year", "f8", ("time"))
    mask = nc.createVariable("mask", "f8", ("y", "x"))
    paper = nc.createVariable("paper", "f8", ("time", "y", "x"))
    country = nc.createVariable("country", "f8", ("time", "y", "x"))
    abstract = nc.createVariable("abstract","f8",("time", "y", "x"))
    participant = nc.createVariable("participant","f8",("time", "y", "x"))
    
    country.missing_value = valex
    paper.missing_value = valex
    abstract.missing_value = valex
    participant.missing_value = valex
    
    lon = np.linspace(0, 10, Nx)
    lat = np.linspace(0, 10, Ny)
    
    year[:] = np.array(Year)
    paper[:] = np.moveaxis(papers_array, [0, 1, 2], [2, 1, 0])
    country[:] = np.moveaxis(country_array, [0, 1, 2], [2, 1, 0])
    abstract[:] = np.moveaxis(abstract_array, [0, 1, 2], [2, 1, 0])
    participant[:] = np.moveaxis(participant_array, [0, 1, 2], [2, 1, 0])
    mask[:] = np.ones((Nx, Ny))